In [1]:
import nibabel as nib
import numpy as np
import surfdist
from surfdist import *

import matplotlib.pyplot as plt
import nilearn.plotting 
import hcp_utils as hcp
import subprocess as sp
from nilearn import signal
from sklearn.metrics import pairwise_distances
from brainspace.gradient import GradientMaps
from utils import *
import mapalign
from mapalign import embed

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [2]:
subj=100206
anatdir=f'/well/margulies/users/mnk884/{subj}/{subj}/Structural/'
fdir=f'/well/margulies/users/mnk884/{subj}/{subj}/Rest'

In [3]:
lcort=slice(0,29696)
rcort=slice(29696, 59412)
cortAll=slice(0,59412)

In [4]:
##### resting state time series paths 
ses1LR=f'{fdir}/rfMRI_REST1_LR_Atlas_hp2000_clean.dtseries.nii'
ses2LR=f'{fdir}/rfMRI_REST2_LR_Atlas_hp2000_clean.dtseries.nii'
ses1RL=f'{fdir}/rfMRI_REST1_RL_Atlas_hp2000_clean.dtseries.nii'
ses2RL=f'{fdir}/rfMRI_REST2_RL_Atlas_hp2000_clean.dtseries.nii'
func_ses=[ses1LR,ses2RL,ses1RL,ses2LR]

In [5]:
#### left anatomical files
Lsrf32=f'{anatdir}/{subj}.L.midthickness.32k_fs_LR.surf.gii'
LsrfNative=f'{anatdir}/{subj}.L.midthickness.native.surf.gii'
Lsphere32=f'{anatdir}/{subj}.L.sphere.32k_fs_LR.surf.gii'
LsphereNat=f'{anatdir}/{subj}.L.sphere.reg.reg_LR.native.surf.gii'
Laparc=f'{anatdir}/{subj}.L.aparc.a2009s.32k_fs_LR.label.gii'

#### right anatomical files 
Rsrf32=f'{anatdir}/{subj}.R.midthickness.32k_fs_LR.surf.gii'
RsrfNative=f'{anatdir}/{subj}.R.midthickness.native.surf.gii'
Rsphere32=f'{anatdir}/{subj}.R.sphere.32k_fs_LR.surf.gii'
RsphereNat=f'{anatdir}/{subj}.R.sphere.reg.reg_LR.native.surf.gii'
Raparc=f'{anatdir}/{subj}.R.aparc.a2009s.32k_fs_LR.label.gii'

In [6]:
func_ses411=[]
for data in range(len(func_ses)):
	#### start by getting the indices of cortical vertices
	func_ses411.append(get_corticalVertices(func_ses[data]))
	##### smooth and clean the funcitonal time series
kernel=5.0 #### smoothed time series kernel

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [7]:
for data in range(len(func_ses)):
	##### smooth and clean the funcitonal time series
	func_ses[data]=wb_smoothCleanTs(func_ses[data],kernel,Lsrf32,Rsrf32)

In [8]:
funcs=[]
for i in func_ses:
    #### extract cortical ROIs
    funcs.append(i.T[cortAll])

del func_ses
data=np.hstack(funcs)

In [9]:
del funcs

In [10]:
tst=np.zeros(32492)
tst[func_ses411[0]['lIDX']]=data.T[0][lcort]

In [11]:
nilearn.plotting.view_surf(Lsrf32,tst)

In [ ]:
# rmat=np.corrcoef(data)

In [ ]:
tst=np.cov(data)

In [ ]:
np.max(tst)

In [ ]:
thr=threshMat(rmat,90)
del rmat


In [ ]:
aff = 1 - pairwise_distances(thr, metric = 'cosine')
del thr

In [ ]:
tst=np.zeros(32492)
tst[func_ses411[0]['lIDX']]=rmat[29000][lcort]
nilearn.plotting.view_surf(Lsrf32,tst)

In [ ]:
grads=DiffEmbed(rmat,3)

In [ ]:
grads